![img](./assets/art29.png)

## 1) 처리할 법 지정

In [22]:
test_article = "제29조"

## 2) 유틸 함수 선언

In [26]:
# ===========================================
# 🧩 Utility functions for law code generation (clean & minimal)
# ===========================================
from pathlib import Path
import json
import re
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

# 파일 읽기 — 담백하게 상대경로 기준
law_path = Path("../../../dataset/PIPA/law/law.json")
with open(law_path, "r", encoding="utf-8") as f:
    data = json.load(f)


# id로 검색 → 해당 조항과 모든 하위 항목을 문자열로 반환
def _extract_article_id(query: str):
    """입력된 문자열에서 '제~조' 형식의 article id 추출"""
    if not isinstance(query, str):
        return None
    match = re.search(r"(제\d+조(?:의\d+)?)", query.strip())
    return match.group(1) if match else None


def format_article_hierarchy(query_id: str):
    """특정 조항(id)과 모든 하위항목을 문자열로 구성"""
    article_id = _extract_article_id(query_id)
    if not article_id:
        return "검색한 문자열에서 '제~조' 형식의 조항 id를 찾을 수 없습니다."

    # 모든 노드를 dict 형태로 구성
    node_by_id = {n["id"]: n for n in data if isinstance(n.get("id"), str)}
    children = {}
    for node in data:
        parent = node.get("parent")
        if isinstance(parent, str):
            children.setdefault(parent, []).append(node["id"])

    # DFS로 하위항목 모두 수집
    def gather(node_id):
        results = []
        for child_id in children.get(node_id, []):
            results.append(child_id)
            results.extend(gather(child_id))
        return results

    if article_id not in node_by_id:
        return f"'{article_id}' 조항을 찾을 수 없습니다."

    target_ids = [article_id] + gather(article_id)
    lines = []
    for tid in target_ids:
        node = node_by_id.get(tid, {})
        var_name = node.get("var_name", "")
        var_part = f" [{var_name}]" if var_name else ""
        content = node.get("content", "")
        lines.append(f"{tid}{var_part} : {content}")
    return "\n".join(lines)


# GPT-5 → JSON 파싱 실패 시 GPT-4o로 재파싱
def parse_with_fallback(output_text: str, prompt: str):
    """GPT-5 결과 JSON 파싱 실패 시 GPT-4o를 이용해 JSON 재생성"""
    try:
        return json.loads(output_text)
    except Exception:
        print("⚠️ GPT-5 결과 JSON 파싱 실패 → GPT-4o로 재시도...")
        messages = [
            {"role": "system", "content": "너는 JSON만 출력하는 법률 코드화 모델이다."},
            {"role": "user", "content": prompt},
        ]
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            response_format={"type": "json_object"},
        )
        return json.loads(completion.choices[0].message.content)

# 자식 수집용 재귀
def gather_children(root_id):
    results = []
    for n in data:
        if n.get("parent") == root_id:
            results.append(n)
            results.extend(gather_children(n["id"]))
    return results

# 제29조와 그 하위 항목들의 리스트 구성
article_id = _extract_article_id(test_article)
node_by_id = {n["id"]: n for n in data if isinstance(n.get("id"), str)}

if article_id in node_by_id:
    root_node = node_by_id[article_id]
    descendants = gather_children(article_id)
    law_subset = [root_node] + descendants
else:
    raise ValueError(f"'{article_id}' 조항을 찾을 수 없습니다.")

print(f"'{article_id}' 및 하위 조항 {len(law_subset)}개 항목 로드 완료.\n")

'제29조' 및 하위 조항 1개 항목 로드 완료.



In [27]:
## 파싱 함수 테스트 
print(format_article_hierarchy("제34조"))

제34조 [LAW_A34] : 
제34조 제1항 [LAW_A34_P1] : 개인정보처리자는 개인정보가 분실ㆍ도난ㆍ유출(이하 이 조에서 “유출등 ”이라 한다)되었음을 알게 되었을 때에는 지체 없이 해당 정보주체에게 다음 각 호의 사항을 알려야 한다. 다만, 정보 주체의 연락처를 알 수 없는 경우 등 정당한 사유가 있는 경우에는 대통령령으로 정하는 바에 따라 통지를 갈음하는 조치를 취할 수 있다.
제34조 제1항 제1호 [LAW_A34_P1_S1] : 유출등이 된 개인정보의 항목
제34조 제1항 제2호 [LAW_A34_P1_S2] : 유출등이 된 시점과 그 경위
제34조 제1항 제3호 [LAW_A34_P1_S3] : 유출등으로 인하여 발생할 수 있는 피해를 최소화하기 위하여 정보주체가 할 수 있는 방법 등에 관한 정보
제34조 제1항 제4호 [LAW_A34_P1_S4] : 개인정보처리자의 대응조치 및 피해 구제절차
제34조 제1항 제5호 [LAW_A34_P1_S5] : 정보주체에게 피해가 발생한 경우 신고 등을 접수할 수 있는 담당부서 및 연락처
제34조 제2항 [LAW_A34_P2] : 개인정보처리자는 개인정보가 유출등이 된 경우 그 피해를 최소화하기 위한 대책을 마련하고 필요한 조치를 하 여야 한다.
제34조 제3항 [LAW_A34_P3] : 개인정보처리자는 개인정보의 유출등이 있음을 알게 되었을 때에는 개인정보의 유형, 유출등의 경로 및 규모 등 을 고려하여 대통령령으로 정하는 바에 따라 제1항 각 호의 사항을 지체 없이 보호위원회 또는 대통령령으로 정하 는 전문기관에 신고하여야 한다. 이 경우 보호위원회 또는 대통령령으로 정하는 전문기관은 피해 확산방지, 피해 복 구 등을 위한 기술을 지원할 수 있다.
제34조 제4항 [LAW_A34_P4] : 제1항에 따른 유출등의 통지 및 제3항에 따른 유출등의 신고의 시기, 방법, 절차 등에 필요한 사항은 대통령령으 로 정한다.


## 3) 기본 변수 선언

In [9]:
BASE_VARIABLES = [
    {
        "variable": "BUSINESS_USES_PERSONAL_INFORMATION",
        "question": "귀사는 고객 또는 이용자의 개인정보를 처리하거나 보유합니까?"
    },
    {
        "variable": "BUSINESS_OUTSOURCES_PROCESSING",
        "question": "귀사는 개인정보 처리 업무를 제3자에게 위탁합니까?"
    }
]

## 4) 법 -> 코드 함수 선언

In [ ]:
# ===========================================
# ⚙️ 단일 조항을 pseudocode JSON으로 변환하는 함수 (content 미포함 + fallback 최소화)
# ===========================================
import json

def generate_single_article_json(node, full_article, BASE_VARIABLES):
    """
    개별 조항(node)을 GPT-5를 이용해 pseudocode JSON으로 변환.

    Parameters
    ----------
    node : dict
        law_subset의 단일 조항 (id, reference 포함)
    full_article : str
        상위 조항 전체 문자열 (format_article_hierarchy(test_article) 결과)
    BASE_VARIABLES : list[dict]
        전역에서 관리되는 누적 business 변수 리스트

    Returns
    -------
    dict : {
        "id": str,
        "gpt_result": dict  # GPT-5 결과 JSON (fallback 포함)
    }
    """
    test_article = node["id"]
    references = node.get("reference", [])

    # 🔍 reference 중 개인정보보호법만 필터링 후 format_article_hierarchy로 변환
    ref_ids = [r["id"] for r in references if r.get("law") == "개인정보보호법"]
    ref_texts = "\n\n".join(
        [format_article_hierarchy(rid) for rid in ref_ids if isinstance(rid, str)]
    )

    fewshot_examples = """
  제16조 제1항: 개인정보처리자는 제15조 제1항 각 호의 어느 하나에 해당하여 개인정보를 수집하는 경우에는 그 목적에 필요한 최소한의 개인정보를 수집하여야 한다.
  {
    "pseudocode": {
      "condition_pseudocode": "received_consent and (LAW_A15_P1_S1['condition'] or LAW_A15_P1_S2['condition'] or LAW_A15_P1_S3['condition'] or LAW_A15_P1_S4['condition'] or LAW_A15_P1_S5['condition'] or LAW_A15_P1_S6['condition'] or LAW_A15_P1_S7['condition'])",
      "legal_pseudocode": "BUSINESS_COLLECTS_MINIMUM_ONLY",
      "action_pseudocode": ""
    },
    "added_variables": [
      {
        "variable": "BUSINESS_COLLECTS_MINIMUM_ONLY",
        "question": "귀사는 고객의 개인정보를 수집할 때 서비스 제공에 반드시 필요한 최소한의 항목만을 수집합니까? 예를 들어 불필요한 생년월일, 주소, 직업, 가족정보 등을 요구하지 않습니까?"
      }
    ]
  }

  제9조 제2항 제1호: 기본계획에는 개인정보 보호의 기본목표와 추진방향이 포함되어야 한다.
  {
    "pseudocode": {
      "condition_pseudocode": "BUSINESS_IS_GOV_AGENCY",
      "legal_pseudocode": "BUSINESS_HAS_PRIVACY_POLICY_GOAL",
      "action_pseudocode": ""
    },
    "added_variables": [
      {
        "variable": "BUSINESS_IS_GOV_AGENCY",
        "question": "귀사의 조직은 공공기관 또는 정부 산하기관입니까?"
      },
      {
        "variable": "BUSINESS_HAS_PRIVACY_POLICY_GOAL",
        "question": "귀사는 개인정보 보호를 위한 목표 및 추진방향을 내부 정책 또는 계획 문서로 수립했습니까?"
      }
    ]
  }

  제2조 제1호: 개인정보’란 살아 있는 개인에 관한 정보로서 성명, 주민등록번호 및 영상 등을 통하여 개인을 식별할 수 있는 정보를 말한다.
  {
    "pseudocode": {
      "condition_pseudocode": "BUSINESS_USES_PERSONAL_INFORMATION",
      "legal_pseudocode": "True",
      "action_pseudocode": ""
    },
    "added_variables": []
  }
"""


    base_var_str = json.dumps(BASE_VARIABLES, ensure_ascii=False, indent=2)
    prompt = f"""
너는 개인정보보호법 각 조항을 논리적으로 코드화하는 모델이다.
입력된 조항을 분석해 아래 구조의 JSON으로 출력하라.

각 조항은 반드시 아래 필드를 포함한다:
- pseudocode
    - condition_pseudocode: 적용 조건 (다른 조항 condition, business 변수 기반)
    - legal_pseudocode : 위법 여부 판단 논리 (False일 때 위반)
    - action_pseudocode: condition_pseudocode가 참일 때 수행되는 조치
- added_variables: 논리를 위해 꼭 필요한 경우에만 추가되는 새 변수
    - 각 변수는 "variable"과 "question"을 반드시 포함
    - question은 실제 비즈니스가 해당 법을 위반했는지 판단할 수 있는 구체적 질문이어야 하며 절대로 법 조항 문구를 인용하거나 설명해서는 안된다.

규칙:
- 모든 법의 각 장, 절, 조, 항, 호는 두 개의 논리 변수를 갖는다.
   - condition_pseudocode : 해당 법 조항이 적용될 수 있는 조건(True/False)
   - legal_pseudocode : 해당 법 조항이 준수되었을 때 True, 위반 시 False
- 모든 변수명은 다음 규칙을 따른다:
   LAW_[조항식별자]['condition' 또는 'legal']
   예: LAW_A29_P1['condition'], LAW_A29_P1['legal']
- 다른 조항을 참조할 때는 반드시 위 변수명을 사용한다.
- 모든 조항의 legal_pseudocode가 False이면 '위반'으로 간주하고, True이면 '준수'로 간주한다.
- condition_pseudocode가 False이면 그 조항은 평가대상(비적용)에서 제외된다.
- 불가피한 경우에만 added_variables를 추가하되, 그 질문은 반드시 실제 비즈니스 행위에 대한 구체적인 질문이어야 한며 절대로 법 조항 문구를 인용하거나 설명해서는 안된다.
- pseudocode 내 논리 연산은 and, or, not을 사용하며, 다른 조항의 condition/legal 변수를 조합해 표현한다.
- 변수명 예시:
   - LAW_A15_P1 : 개인정보보호법 제15조 제1항
   - LAW_A15_P1_S2 : 개인정보보호법 제15조 제1항 제2호
   - DECREE_A28_P3 : 개인정보보호법 시행령 제28조 제3항
- action_pseudocode는 조항 간 영향을 코드화할 때만 사용한다.
    
다음은 개인정보보호법 조항을 JSON으로 코드화한 예시들이다.
이때, legal_pseudocode가 False일 경우 '위반'으로 간주한다. (True → 준수)

[예제]
{fewshot_examples}

아래는 비즈니스 평가를 위한 기본 변수 목록이다.
이 변수들은 논리 구성에 우선 활용되어야 하며, 불가피한 경우에만 새 변수를 추가하라.

[기본 변수 목록]
{base_var_str}

[전체 조항]
{full_article}

[처리해야하는 조항]
{test_article}

[레퍼런스]
{ref_texts}
"""

    # 🧠 GPT-5 호출
    try:
        response = client.responses.create(
            model="gpt-5",
            input=prompt,
            reasoning={"effort": "low"},
        )
        output_text = response.output_text

        # ⚙️ GPT-5 결과를 직접 JSON으로 파싱 시도
        try:
            result = json.loads(output_text)
        except json.JSONDecodeError:
            # ⚠️ 실패 시 fallback
            print(f"⚠️ {test_article}: GPT-5 결과 JSON 파싱 실패 → fallback 사용")
            result = parse_with_fallback(output_text, prompt)

    except Exception as e:
        print(f"❌ '{test_article}' 처리 중 오류: {e}")
        result = {"id": test_article, "error": str(e)}

    return {
        "id": test_article,
        "gpt_result": result
    }


## 5) 메인 루프

In [ ]:
# ===========================================
# ⚙️ 메인 루프: 모든 law_subset 조항 반복 처리
# ===========================================
from tqdm import tqdm

# BASE_VARIABLES는 전역에 존재 (누적 관리)
# law_subset은 이미 제29조 및 하위 조항들이 포함된 리스트
# full_article은 format_article_hierarchy("제29조") 결과 문자열

for node in tqdm(law_subset, desc="🔄 Generating pseudocode with GPT-5"):
    # 1️⃣ GPT-5 호출 및 JSON 변환 수행
    result = generate_single_article_json(node, full_article, BASE_VARIABLES)

    # 2️⃣ gpt_result를 현재 node에 추가
    node["gpt_result"] = result.get("gpt_result", {})

    # 3️⃣ gpt_result 내부에서 added_variables 추출 후 전역 BASE_VARIABLES에 누적
    try:
        gpt_result = result.get("gpt_result", {})
        # gpt_result가 전체 조항 JSON을 감싸는 경우 (예: {"제29조 제1항": {...}})
        if isinstance(gpt_result, dict):
            # 하나의 조항 키를 찾아 내부 added_variables 탐색
            for key, val in gpt_result.items():
                if isinstance(val, dict) and "added_variables" in val:
                    for v in val["added_variables"]:
                        # 중복 방지: 동일 variable명 없을 때만 추가
                        if not any(x["variable"] == v["variable"] for x in BASE_VARIABLES):
                            BASE_VARIABLES.append(v)
    except Exception as e:
        print(f"⚠️ {node['id']} 변수 누적 중 오류 발생: {e}")

print(f"\n✅ 전체 {len(law_subset)}개 조항 처리 완료!")
print(f"📦 BASE_VARIABLES 누적 개수: {len(BASE_VARIABLES)}\n")
